In [6]:
import pandas as pd

# Cargar los tres archivos
vtm = pd.read_csv("data/udm_vtm_melted_mapping.csv")
mt = pd.read_csv("data/mt_consolidated_pe.csv")
pred = pd.read_csv("data/prediction_input_series.csv")  # si ya lo tienes local

# Normalizar nombres
vtm["SOURCE_SKU"] = vtm["SOURCE_SKU"].astype(str).str.upper().str.strip()
mt["PPG"] = mt["PPG"].astype(str).str.upper().str.strip()
pred["SKU_GROUP"] = pred["SKU_GROUP"].astype(str).str.upper().str.strip()

# Crear conjuntos
vtm_skus = set(vtm["SOURCE_SKU"].unique())
mt_skus = set(mt["PPG"].unique())
pred_skus = set(pred["SKU_GROUP"].unique())

# Comparaciones
solo_vtm = vtm_skus - (mt_skus | pred_skus)
solo_mt = mt_skus - (vtm_skus | pred_skus)
solo_pred = pred_skus - (vtm_skus | mt_skus)
en_todos = vtm_skus & mt_skus & pred_skus

print(f"SKUs solo en VTM: {len(solo_vtm)}")
print(f"SKUs solo en Consolidated: {len(solo_mt)}")
print(f"SKUs solo en Prediction: {len(solo_pred)}")
print(f"SKUs en las tres fuentes: {len(en_todos)}")


SKUs solo en VTM: 158
SKUs solo en Consolidated: 14
SKUs solo en Prediction: 2
SKUs en las tres fuentes: 53


In [7]:
import pandas as pd

# Cargar datos
vtm = pd.read_csv("data/udm_vtm_melted_mapping.csv")

# Normalizar fabricantes
vtm["SRC_MFG"] = vtm["SRC_MFG"].astype(str).str.upper().str.strip()
vtm["DEST_MFG"] = vtm["DEST_MFG"].astype(str).str.upper().str.strip()

# Filtrar canibalización interna
cani = vtm[
    (vtm["SRC_MFG"].str.contains("COCA", na=False)) &
    (vtm["DEST_MFG"].str.contains("COCA", na=False)) &
    (vtm["VOLUME_TRANSFER"] > 0)
].copy()

# Agregar la pérdida total de volumen por SKU origen
cani_summary = (
    cani.groupby("SOURCE_SKU", dropna=False)["VOLUME_TRANSFER"]
    .sum(min_count=1)
    .reset_index()
    .sort_values("VOLUME_TRANSFER", ascending=False)
)

# Renombrar columna
cani_summary = cani_summary.rename(columns={"VOLUME_TRANSFER": "VTM_CANIBALIZACION"})

print(f"SKUs Coca-Cola con canibalización interna detectados: {len(cani_summary)}")
cani_summary.head(20)


SKUs Coca-Cola con canibalización interna detectados: 58


,SOURCE_SKU,VTM_CANIBALIZACION
21,COCA-COLA SIN AZUCAR PET NR 2500,100.001
25,COCA-COLA SIN AZUCAR REF PET RETORNABLE 2000,100.001
40,PREMIO PET NR 1750,100.000
47,ROMAN PET NR 3000,100.000
9,COCA-COLA LATA NR 235,100.000
17,COCA-COLA SIN AZUCAR LATA NR 235,100.000
34,DEL VALLE TETRAPACK NR 250,100.000
53,SPRITE PET NR 1250,100.000
26,COCA-COLA VIDRIO NR 300,99.999
20,COCA-COLA SIN AZUCAR PET NR 250,99.999


In [10]:
import pandas as pd

# Cargar tus dos archivos
vtm_canibal = pd.read_csv("data/udm_vtm_melted_mapping.csv")
pred = pd.read_csv("data/prediction_input_series.csv")

# --- Normalizar texto para coincidencia robusta ---
def normalize(s):
    return s.astype(str).str.strip().str.upper()

# Canibalización interna (según tus resultados)
canib_skus = [
    "COCA-COLA SIN AZUCAR PET NR 2500",
    "COCA-COLA SIN AZUCAR REF PET RETORNABLE 2000",
    "PREMIO PET NR 1750",
    "ROMAN PET NR 3000",
    "COCA-COLA LATA NR 235",
    "COCA-COLA SIN AZUCAR LATA NR 235",
    "DEL VALLE TETRAPACK NR 250",
    "SPRITE PET NR 1250",
    "COCA-COLA VIDRIO NR 300",
    "COCA-COLA SIN AZUCAR PET NR 250",
    "COCA-COLA SIN AZUCAR PET NR 1750",
    "DEL VALLE PET NR 400",
    "MANANTIAL PET NR 500",
    "COCA-COLA PET NR 250",
    "COCA-COLA PET NR 600",
    "COCA-COLA PET NR 2500",
    "COCA-COLA SIN AZUCAR PET NR 600",
    "COCA-COLA SIN AZUCAR PET NR 3000",
    "SPRITE PET NR 1750",
    "COCA-COLA PET NR 1750"
]

canib_df = pd.DataFrame({"SOURCE_SKU": canib_skus})
canib_df["SOURCE_SKU"] = normalize(canib_df["SOURCE_SKU"])
pred["SKU_GROUP"] = normalize(pred["SKU_GROUP"])

# --- Cruce ---
canib_df["En_prediction_input_series"] = canib_df["SOURCE_SKU"].isin(pred["SKU_GROUP"])

# --- Separar por estado ---
presentes = canib_df[canib_df["En_prediction_input_series"] == True]
faltantes = canib_df[canib_df["En_prediction_input_series"] == False]

print(f"SKUs de Coca-Cola con canibalización interna encontrados en prediction_input_series: {len(presentes)}")
print(f"SKUs que no aparecen en prediction_input_series: {len(faltantes)}")

print("\n✅ Coinciden:")
print(presentes["SOURCE_SKU"].to_list())

print("\n⚠️ No encontrados:")
print(faltantes["SOURCE_SKU"].to_list())


SKUs de Coca-Cola con canibalización interna encontrados en prediction_input_series: 16
SKUs que no aparecen en prediction_input_series: 4

✅ Coinciden:
['COCA-COLA SIN AZUCAR PET NR 2500', 'COCA-COLA SIN AZUCAR REF PET RETORNABLE 2000', 'PREMIO PET NR 1750', 'ROMAN PET NR 3000', 'SPRITE PET NR 1250', 'COCA-COLA VIDRIO NR 300', 'COCA-COLA SIN AZUCAR PET NR 250', 'COCA-COLA SIN AZUCAR PET NR 1750', 'DEL VALLE PET NR 400', 'COCA-COLA PET NR 250', 'COCA-COLA PET NR 600', 'COCA-COLA PET NR 2500', 'COCA-COLA SIN AZUCAR PET NR 600', 'COCA-COLA SIN AZUCAR PET NR 3000', 'SPRITE PET NR 1750', 'COCA-COLA PET NR 1750']

⚠️ No encontrados:
['COCA-COLA LATA NR 235', 'COCA-COLA SIN AZUCAR LATA NR 235', 'DEL VALLE TETRAPACK NR 250', 'MANANTIAL PET NR 500']


In [12]:
import pandas as pd

# Cargar el archivo
vtm = pd.read_csv("data/udm_vtm_melted_mapping.csv")

# Normalizar columnas clave
vtm["SRC_MFG"] = vtm["SRC_MFG"].astype(str).str.upper()
vtm["DEST_MFG"] = vtm["DEST_MFG"].astype(str).str.upper()
vtm["SOURCE_SKU"] = vtm["SOURCE_SKU"].astype(str).str.strip().str.upper()
vtm["DESTINATION_SKU"] = vtm["DESTINATION_SKU"].astype(str).str.strip().str.upper()

# Filtrar canibalización interna: Coca-Cola → Coca-Cola
canibal = vtm[
    vtm["SRC_MFG"].str.contains("COCA", na=False) &
    vtm["DEST_MFG"].str.contains("COCA", na=False) &
    (pd.to_numeric(vtm["VOLUME_TRANSFER"], errors="coerce") > 0)
].copy()

# Agregar volumen total por SKU origen
canibal_summary = (
    canibal.groupby("SOURCE_SKU", dropna=False)["VOLUME_TRANSFER"]
    .sum()
    .reset_index()
    .sort_values("VOLUME_TRANSFER", ascending=False)
)

canibal_summary.rename(columns={"SOURCE_SKU": "SKU_CANIBALIZADO", "VOLUME_TRANSFER": "VTM_CANIBALIZACION"}, inplace=True)

# Mostrar los SKUs más canibalizados
canibal_summary.head(20)


,SKU_CANIBALIZADO,VTM_CANIBALIZACION
21,COCA-COLA SIN AZUCAR PET NR 2500,100.001
25,COCA-COLA SIN AZUCAR REF PET RETORNABLE 2000,100.001
40,PREMIO PET NR 1750,100.000
47,ROMAN PET NR 3000,100.000
9,COCA-COLA LATA NR 235,100.000
17,COCA-COLA SIN AZUCAR LATA NR 235,100.000
34,DEL VALLE TETRAPACK NR 250,100.000
53,SPRITE PET NR 1250,100.000
26,COCA-COLA VIDRIO NR 300,99.999
20,COCA-COLA SIN AZUCAR PET NR 250,99.999


In [15]:
import pandas as pd

vtm = pd.read_csv("data/udm_vtm_melted_mapping.csv")
pred = pd.read_csv("data/prediction_input_series.csv")
mt = pd.read_csv("data/mt_consolidated_pe.csv")

# Normalizar nombres
def norm(x): return x.astype(str).str.upper().str.strip()
vtm["SOURCE_SKU"] = norm(vtm["SOURCE_SKU"])
pred["SKU_GROUP"] = norm(pred["SKU_GROUP"])
mt["PPG"] = norm(mt["PPG"])

sku = "COCA-COLA SIN AZUCAR PET NR 2500"

print("En VTM:", sku in vtm["SOURCE_SKU"].values)
print("En Prediction:", sku in pred["SKU_GROUP"].values)
print("En Consolidated:", sku in mt["PPG"].values)

En VTM: True
En Prediction: True
En Consolidated: True


In [17]:
from pathlib import Path
import pandas as pd

p = Path("data")

pred = pd.read_csv(p/"prediction_input_series.csv")
mt = pd.read_csv(p/"mt_consolidated_pe.csv")

# Normalizar para evitar diferencias invisibles
pred["SKU_NORM"] = pred["SKU_GROUP"].astype(str).str.upper().str.strip()
mt["SKU_NORM"] = mt["PPG"].astype(str).str.upper().str.strip()

# Calcular volumen agregado (últimos 24 meses)
pred["PERIOD"] = pd.to_datetime(pred["PERIOD"], errors="coerce")
win_end = pred["PERIOD"].max()
win_start = win_end - pd.offsets.DateOffset(months=24)
vol_df = (
    pred[(pred["PERIOD"] >= win_start) & (pred["PERIOD"] < win_end)]
    .groupby("SKU_NORM")["UC_CALCULATED"]
    .sum()
    .rename("W_CU")
    .reset_index()
)

merged = mt.merge(vol_df, on="SKU_NORM", how="left")

print(merged[merged["SKU_NORM"].str.contains("SIN AZUCAR PET NR 2500", case=False)])


   Channel                               PPG CATEGORY SUB_CATEGORY  \
19      MT  COCA-COLA SIN AZUCAR PET NR 2500     CSDs        COLAS   

       MARCA_REVISED_UPD    PPG_PE  VTM_RATIO    NET_PE  \
19  Coca-Cola Sin Azucar -2.052646    1.00001 -2.052667   

                            SKU_NORM  W_CU  
19  COCA-COLA SIN AZUCAR PET NR 2500   NaN  


/var/folders/yh/tx0ldqx102s99q8tjd118fgh0000gn/T/ipykernel_11890/1550795608.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred["PERIOD"] = pd.to_datetime(pred["PERIOD"], errors="coerce")


In [3]:
import pandas as pd
import re

# Cargar el archivo
df = pd.read_csv("data/udm_vtm_melted_mapping.csv")

# --- Limpieza robusta de DESTINATION_SKU ---
def clean_sku(s):
    if not isinstance(s, str):
        return ""
    s = s.upper().strip()
    # Sustituir separadores comunes por espacios
    s = s.replace("-", " ").replace("_", " ").replace(".", " ")
    # Eliminar caracteres especiales no alfanuméricos
    s = re.sub(r"[^A-Z0-9ÁÉÍÓÚÑ ]+", " ", s)
    # Normalizar espacios múltiples
    s = re.sub(r"\s+", " ", s)
    return s.strip()

df["DESTINATION_SKU"] = df["DESTINATION_SKU"].astype(str).apply(clean_sku)
df["SRC_MFG"] = df["SRC_MFG"].astype(str).str.upper().str.strip()
df["DEST_MFG"] = df["DEST_MFG"].astype(str).str.upper().str.strip()
df["REVISED_CATEGORY"] = df["REVISED_CATEGORY"].astype(str).str.upper().str.strip()
df["VOLUME_TRANSFER"] = pd.to_numeric(df["VOLUME_TRANSFER"], errors="coerce").fillna(0)

# --- Filtro con condiciones robustas ---
mask = (
    (df["SRC_MFG"] == "COCA COLA FABRICANTE") &
    (df["DEST_MFG"] == "COCA COLA FABRICANTE") &
    (df["REVISED_CATEGORY"] == "AGUA") &
    (df["VOLUME_TRANSFER"] > 0)
)

# --- Obtener SKUs destino únicos limpios ---
unique_skus_destino = sorted(df.loc[mask, "DESTINATION_SKU"].dropna().unique())

print(f"✅ Total SKUs destino únicos: {len(unique_skus_destino)}")
for sku in unique_skus_destino:
    print(" -", sku)



✅ Total SKUs destino únicos: 13
 - BRISA CON GAS PET NR 600
 - BRISA GARAFFON NR 6000
 - BRISA PET NR 1000
 - BRISA PET NR 280
 - BRISA PET NR 3000
 - BRISA PET NR 600
 - BRISA SABORIZADA PET NR 1500
 - BRISA SABORIZADA PET NR 280
 - BRISA SABORIZADA PET NR 600
 - MANANTIAL CON GAS PET NR 600
 - MANANTIAL PET NR 1500
 - MANANTIAL PET NR 500
 - MANANTIAL PET NR 600


In [8]:
import pandas as pd
import re

# --- SKUs de origen a analizar ---
ORIGINS = [
    "BRISA SABORIZADA PET NR 1500",
    "BRISA SABORIZADA PET NR 280",
    "BRISA SABORIZADA PET NR 600",
]

# --- Limpieza de texto ---
def _norm(s):
    if not isinstance(s, str):
        return ""
    s = s.upper().strip()
    s = re.sub(r"\s+", " ", s)
    return s

# --- Cargar archivo ---
df = pd.read_csv("data/udm_vtm_melted_mapping.csv")

# Normalizar columnas
for c in ["SRC_MFG", "SOURCE_SKU", "DEST_MFG", "DESTINATION_SKU"]:
    if c in df.columns:
        df[c] = df[c].astype(str).map(_norm)

df["VOLUME_TRANSFER"] = pd.to_numeric(df["VOLUME_TRANSFER"], errors="coerce").fillna(0)

# Filtrar por fabricante Coca-Cola, volumen positivo y SKUs objetivo
df = df[
    (df["SRC_MFG"] == "COCA COLA FABRICANTE") &
    (df["VOLUME_TRANSFER"] > 0) &
    (df["SOURCE_SKU"].isin([_norm(x) for x in ORIGINS]))
]

# 🔥 Deduplicar combinaciones reales SKU origen → SKU destino (sin considerar periodo, canal, etc.)
df = df.drop_duplicates(subset=["SOURCE_SKU", "DESTINATION_SKU"])

# Conteo global
skus_origen_unicos = df["SOURCE_SKU"].nunique()
skus_destino_unicos = df["DESTINATION_SKU"].nunique()

print(f"SKUs únicos de origen (debe ser 3): {skus_origen_unicos}")
print(f"SKUs únicos de destino (deben ser 17): {skus_destino_unicos}\n")

# Agrupar por fabricante destino
res = (
    df.groupby("DEST_MFG", dropna=False)
      .agg(
          skus_origen_unicos=("SOURCE_SKU", "nunique"),
          skus_destino_unicos=("DESTINATION_SKU", "nunique"),
      )
      .reset_index()
      .sort_values("skus_destino_unicos", ascending=False)
)

print(res.to_string(index=False))


SKUs únicos de origen (debe ser 3): 3
SKUs únicos de destino (deben ser 17): 27

            DEST_MFG  skus_origen_unicos  skus_destino_unicos
COCA COLA FABRICANTE                   3                   19
               OTRAS                   2                    3
 POSTOBON FABRICANTE                   3                    3
               QUALA                   2                    2
